# What You Got Is What You Got| PyData LA 2019
> Composition, inheritance, restricted computation domains, boxed versus unboxed, and the search for a perfect proxy. But, folks, what you got is what you got.

We are very familiar with packages like numpy, pandas, networkx and more specifically these packages provide us with a lot of mechanisms and algorithms that we might use but mosT specifically we are using ndarray, DataFrame, DiGraph respectively. Now one question that we ask is what are these things really. 

We might call these things **ADT(Abstract Data Types)**. But when we think of these as ADT in the context of python we may think of tuple, list, dict. Theses built-in data types have some meaning associated with them like dict is a mapping. Similarly ndarray is a n-dim array, dataframe is a columar panel, DiGrah is a tree. But one thing that cones to our mind when we think of ADT in python is that they have small API. Like for a set in python me may think of small set of operations like union, intersection and some more. Similarly for a dictionary we may thing of contains, setitem, getitem, delitem. But when we see at ndarray, DataFrame, Digraph we can see that the surface area is larger than what you assume for ADT's.

So we might think of them as something being **restricted computation domain**. When we are talking about restricted computation domains we end up seeing a very common pattern when we're writing python code. Often time when we are looking at some code we are looking at some entity (a python class) and me might have lots of these.

In [ ]:
class BusinessObject:
    pass

# We might have lots of these
[BusinessObject() for _ in range(100_000_000)]

When you have lots of these you start to see serious performance problems. This is the consequence of the python object being too big, not being cache coherent and having unbounded computational complexity. So you can now try resolving them.

Python objects are too big because they carry this dict around them everywhere, so I can address this issue by adding slots and this will shrink the size of individual python object. But they are not very interesting.

In [ ]:
class T:
    pass

assert hasattr(T, '__dict__')

In [ ]:
class T:
    __slots__ = ()

Python objects are not cache coherent when there are many python objects floating around a system. The solution to this is you can have a custom allocator because nothing in python says how things need to be allocated. You can solve this problem.

But the much bigger problem is the unbounded computational complexity of a python object. And this is what necessitates the need of a restricted computation domain. In other words you might have a python object that has an add method and you perform some operation on it and it just works.

In [ ]:
class T:
    def __init__(self, value, *deps):
        self.value, self.deps = value, deps
    def __add__(self, other):
        for d in self.deps:
            d.value += 1
        return T(self.value + other.value)
    
x = T(1)
y = T(2, x)
z = T(3, x, y)

sum([x, y, z], T(0))

But if you think about that object you can add unbounded complexity to the internals of that object. For example in this particular case each one of these items has some hidden dependency that won't be visible to the actual container structure or to the sum function and when you do something like this you start to break some assumptions that your container type is to have as to how these operations will be performed, like they are independent of each other. This case breaks all those assumptions.

So you might say that numpy.ndarray is a very nice solution to this problem as it is contiguous data, unboxed data and machine types. Because these are machine types so there are no hidden inter-dependencies. So when you look at the code that uses a pandas dataframe of a numpy ndarray if fairly follows a very simple pattern. Pattern looks this, you load the data in python, maybe do some cleaning of the data in python, then you put that into a computation domain and perform some operations in the computation domain and at the end you get the result out in pyhton.

In [ ]:
data = load_data()
cleaned_data = [clean(d) for d in data]
df = DataFrame(cleaned_data)
df.op()
df.op()
df.op()
df.plot()

Inside the domain the data is managed, so the domain can set some rules about what happens in order to cross that boundary to take raw python object and put them into the domain. Once they are in that domain you can do whatever you want, everything is handled by the domain. 

What this suggest though is that there is no natural bound on the API of the surface area of the restricted computation domain. As the domain has to handle all the possible ways you can tweak the data in that domain. So this API is going to be much larger.

It turns out that when you are using numpy.ndarrys you simply create an array by calling the array method, same for pandas and networkx. However it is often the case that these entities need to be situated in some production or business context. In other words lets say that you're using a networkx digraph but you are not using the networkx digraph in order to do abstract academic graph algorithm exercises but you are actually trying to do something like perform point-to-point OSR computation for a mesh optical network. In that case you need to have some representation between the underlying data structure and the entities that it's manipulating and you certainly don't want to do this, where you say I am going to figure out what is the OSR between two transceivers and 0 represents Los Angeles and 1 represents San Francisco.

In [ ]:
from networkx import DiGraph
g = DiGraph()
g.add_edge(0, 1) # 0 = Los Angeles
                 # 1 = San Francisco

And you definitely don't want to make the numbers strings instead.

In [ ]:
g = DiGraph()
g.add_edge('Los Angeles', 
           'San Francisco')

Instead you have some custom object and that custom object will have the data that will not be used in the computation and you put those into your digraph.

In [ ]:
class Transceiver:
    def __init__(self, label):
        self.label = label

g = DiGraph()
g.add_edge(Transceiver('Los Angeles'),
           Transceiver('San Francisco'))

Now the custom data may contain information that's not used in the computation like coords for better plotting.

In [ ]:
class Transceiver:
    def __init__(self, lable, coords):
        self.label = labels
        self.coords = coords